In [25]:
import pandas_datareader as web
import pandas as pd
from yahoo_fin import stock_info as si 
import datetime as dt
import yfinance as yf

In [16]:
tickers = si.tickers_sp500()

In [17]:
start = dt.datetime.now() - dt.timedelta(days=365)
end = dt.datetime.now()

In [31]:
spx_df = yf.download(tickers=['^GSPC'], start=start, end=end)
spx_df['pct_change'] = spx_df['Adj Close'].pct_change()
spx_return = (spx_df['pct_change'] + 1).cumprod()[-1]

[*********************100%***********************]  1 of 1 completed


In [32]:
spx_return

1.019165006115376

In [33]:
spx_df.head()

,Open,High,Low,Close,Adj Close,Volume,pct_change
Date,,,,,,,
2022-06-01,4149.779785,4166.540039,4073.850098,4101.229980,4101.229980,4531800000,NaN
2022-06-02,4095.409912,4177.509766,4074.370117,4176.819824,4176.819824,4405790000,0.018431
2022-06-03,4137.569824,4142.669922,4098.669922,4108.540039,4108.540039,3711110000,-0.016347
2022-06-06,4134.720215,4168.779785,4109.180176,4121.430176,4121.430176,4332700000,0.003137
2022-06-07,4096.470215,4164.859863,4080.189941,4160.680176,4160.680176,4248210000,0.009523


In [34]:
return_list = []
final_df = pd.DataFrame(columns=['ticker','px_last','score','pe_ratio','peg_ratio','sma_150','sma_200','52week_low','52week_high'])

In [56]:
#counter = 0
for ticker in tickers:
    tempdf = yf.download(tickers=['^GSPC'], start=start, end=end)
    tempdf.to_csv(f'./{ticker}.csv')
    
    tempdf['pct_change'] = tempdf['Adj Close'].pct_change()
    
    stock_return = (tempdf['pct_change'] + 1).cumprod()[-1]
    
    returns_compared = round((stock_return / spx_return), 2)
    return_list.append(returns_compared)
    
    # counter += 1
    # if counter == 10:
    #     break
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [72]:
top_performers = pd.DataFrame(list(zip(tickers, return_list)), columns =['ticker','returns_compared'])
top_performers['score'] = top_performers['returns_compared'].rank(pct=True)*100
top_performers[top_performers['score'] >= top_performers['score'].quantile(0.8)]    #top 20% historically
top_performers['ticker'].nunique()

503

In [70]:
for ticker in top_performers['ticker']:
    try:
        df = pd.read_csv(f'./{ticker}.csv', index_col=0)
        sma = [150, 200]
        for ma in sma:
            df['sma_' + str(ma)] = round(df['Adj Close'].rolling(window=ma).mean(), 2)
        
        px_last = df['Adj Close'][-1]
        pe_ratio = float(si.get_quote_table(ticker)['PE Ratio (TTM)'])
        peg_ratio = float(si.get_stats_valuation(ticker)[1][4])
        sma_150 = df['sma_150'][-1]
        sma_200 = df['sma_200'][-1]
        low_52week = round(min(df['Low'][-(52*5):]),2)  #better than just 365 for trading days
        high_52week= round(max(df['High'][-(52*5):]),2)
        score = round(top_performers[top_performers['ticker'] == ticker]['score'].tolist()[0])
        
        # conditions
        
        condition_1 = px_last > sma_150 > sma_200   # last price has to be larger than sma150 and sma 150 larger than sma 200
        condition_2 = px_last >= (1.3 * low_52week) # last price 30% higher than 52 week low
        condition_3 = px_last >= (0.75 * high_52week)   # last price 75% higher than 52 week high
        condition_4 = pe_ratio < 40
        condition_5 =  peg_ratio < 2
        
        if condition_1 and condition_2 and condition_3 and condition_4 and condition_5:
            final_df = final_df.append({'ticker': ticker
                                        ,'px_last': px_last
                                        ,'score': score
                                        ,'pe_ratio': pe_ratio
                                        ,'peg_ratio': peg_ratio
                                        ,'sma_150': sma_150
                                        ,'sma_200': sma_200
                                        ,'52week_low': low_52week
                                        ,'52week_high': high_52week}, ignore_index=True)
    except Exception as e:
        print(f"{e} for {ticker}")
        
final_df.sort_values(by='score', ascending=False)
pd.set_option('display.max_columns',10)

print(final_df)
final_df.to_csv('final.csv')
    
#print(si.get_stats_valuation)

/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)


1 for A


/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)


1 for AAL


/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)


...

1 for ZBRA


/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)


1 for ZION


/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:295: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = tables[0].append(tables[1])
/home/creeg/.local/lib/python3.10/site-packages/yahoo_fin/stock_info.py:302: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(quote_price)


1 for ZTS
Empty DataFrame
Columns: [ticker, px_last, score, pe_ratio, peg_ratio, sma_150d, sma_200d, 52week_low, 52week_high]
Index: []
